# Implementation of GloVe 
[source](http://www.foldl.me/2014/glove-python/)

GloVe (Global Vectors for Word Representation) is a tool released by Stanford NLP Group researchers Jeffrey Pennington, Richard Socher, and Chris Manning for learning continuous-space vector representations of words.

The GloVe model learns word vectors by examining word co-occurrences within a text corpus. Before we train the actual model, we need to construct a co-occurrence matrix X, where a cell Xij is a “strength” which represents how often the word i appears in the context of the word j. We run through our corpus just once to build the matrix X, and from then on use this co-occurrence data in place of the actual corpus. We will construct our model based only on the values collected in X.

Once we’ve prepared X, our task is to decide vector values in continuous space for each word we observe in the corpus. We will produce vectors with a soft constraint that for each word pair of word i and word j,

<img src="files/glove_eq.png"/>

where bi and bj are scalar bias terms associated with words i and j, respectively. Intuitively speaking, we want to build word vectors that retain some useful information about how every pair of words i and j co-occur.

We’ll do this by minimizing an objective function J, which evaluates the sum of all squared errors based on the above equation, weighted with a function f:

<img src="files/glove_eq2.png"/>

We choose an f that helps prevents common word pairs (i.e., those with large Xij values) from skewing our objective too much:



<img src="files/glove_eq3.png">

When we encounter extremely common word pairs (where Xij>xmax) this function will cut off its normal output and simply return 1. For all other word pairs, we return some weight in the range (0,1), where the distribution of weights in this range is decided by α.



# Data Preparation

In [14]:
from nltk.corpus import brown
import operator
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [46]:
KEEP_WORDS = set([
  'king', 'man', 'queen', 'woman',
  'italy', 'rome', 'france', 'paris',
  'london', 'britain', 'england',
])

In [47]:
n_vocab=5000
keep_words=KEEP_WORDS
sentences = brown.sents()
indexed_sentences = []

In [48]:
i = 2
word2idx = {'START': 0, 'END': 1}
idx2word = ['START', 'END']

word_idx_count = {
    0: float('inf'),
    1: float('inf'),
  }

for sentence in sentences:
    indexed_sentence = []
    for token in sentence:
        token = token.lower()
        if token not in word2idx:
            idx2word.append(token)
            word2idx[token] = i
            i += 1

      # keep track of counts for later sorting
        idx = word2idx[token]
        word_idx_count[idx] = word_idx_count.get(idx, 0) + 1

        indexed_sentence.append(idx)
    indexed_sentences.append(indexed_sentence)

In [49]:
  # restrict vocab size

  # set all the words I want to keep to infinity
  # so that they are included when I pick the most
  # common words
for word in keep_words:
    word_idx_count[word2idx[word]] = float('inf')

sorted_word_idx_count = sorted(word_idx_count.items(), key=operator.itemgetter(1), reverse=True)
word2idx_small = {}
new_idx = 0
idx_new_idx_map = {}
for idx, count in sorted_word_idx_count[:n_vocab]:
    word = idx2word[idx]
    print(word, count)
    word2idx_small[word] = new_idx
    idx_new_idx_map[idx] = new_idx
    new_idx += 1
  # let 'unknown' be the last token
word2idx_small['UNKNOWN'] = new_idx 
unknown = new_idx

assert('START' in word2idx_small)
assert('END' in word2idx_small)
for word in keep_words:
    assert(word in word2idx_small)

  # map old idx to new idx
sentences_small = []
for sentence in indexed_sentences:
    if len(sentence) > 1:
        new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
        sentences_small.append(new_sentence)

START inf
END inf
man inf
paris inf
britain inf
england inf
king inf
woman inf
rome inf
london inf
queen inf
italy inf
france inf
the 69971
, 58334
. 49346
of 36412
and 28853
to 26158
a 23195
in 21337
that 10594
is 10109
was 9815
he 9548
for 9489
`` 8837
'' 8789
it 8760
with 7289
as 7253
his 6996
on 6741
be 6377
; 5566
at 5372
by 5306
i 5164
this 5145
had 5133
? 4693
not 4610
are 4394
but 4381
from 4370
or 4206
have 3942
an 3740
they 3620
which 3561
-- 3432
one 3292
you 3286
were 3284
her 3036
all 3001
she 2860
there 2728
would 2714
their 2669
we 2652
him 2619
been 2472
) 2466
has 2437
( 2435
when 2331
who 2252
will 2245
more 2215
if 2198
no 2139
out 2097
so 1985
said 1961
what 1908
up 1890
its 1858
about 1815
: 1795
into 1791
than 1790
them 1788
can 1772
only 1748
other 1702
new 1635
some 1618
could 1601
time 1598
! 1596
these 1573
two 1412
may 1402
then 1380
do 1363
first 1361
any 1344
my 1318
now 1314
such 1303
like 1292
our 1252
over 1236
me 1181
even 1170
most 1159
made 1125
also 

request 49
firmly 49
count 49
hopes 49
tendency 49
acceptance 49
driver 49
depends 49
ride 49
impressive 49
sports 49
milk 49
quietly 49
holy 49
tragedy 49
burning 49
incident 49
operator 49
payments 49
creative 49
silent 49
measures 49
consideration 49
till 49
leaves 49
partly 49
grand 48
suit 48
destroy 48
24 48
co. 48
hoped 48
royal 48
limit 48
operate 48
twelve 48
guard 48
integration 48
tired 48
1957 48
screen 48
mantle 48
charlie 48
shooting 48
she's 48
cry 48
via 48
pink 48
mile 48
missile 48
functions 48
formal 48
occasionally 48
rolled 48
comparison 48
resistance 48
personality 48
concrete 48
precisely 48
plain 48
swung 48
sorry 48
maintained 48
drinking 48
intelligence 48
anger 48
poem 48
attitudes 48
liquid 48
hearst 48
considering 47
bonds 47
denied 47
bills 47
employment 47
cook 47
grant 47
fears 47
21 47
cuba 47
sold 47
thousands 47
manufacturers 47
engaged 47
provision 47
purchase 47
safety 47
honest 47
representative 47
deny 47
northern 47
moscow 47
expenses 47
expansio

indication 20
tones 20
cafe 20
hasn't 20
trap 20
figured 20
abel 20
reactionary 20
tent 20
blonde 20
lobby 20
renaissance 20
responded 20
founded 20
fantastic 20
binding 20
lean 20
landscape 20
amazing 20
alike 20
passages 20
reporter 20
cooperative 20
alabama 20
happily 20
shadows 20
rises 20
fortunately 20
vacuum 20
endless 20
minority 20
overwhelming 20
jungle 20
convictions 20
nest 20
fascinating 20
accordance 20
motives 20
listeners 20
distinctive 20
tooth 20
attain 20
styles 20
bones 20
wit 20
solely 20
socialism 20
grains 20
sixteen 20
hatred 20
creatures 20
biological 20
poverty 20
twentieth 20
historians 20
manage 20
authentic 20
laura 20
dolores 20
leaped 20
doc 20
transformation 20
theories 20
worries 20
merchant 20
christ's 20
b.c. 20
anyhow 20
relating 20
probabilities 20
impulse 20
package 20
pupil 20
anticipation 20
slide 20
fractions 20
cathy 20
boots 20
sauce 20
mustard 20
michelangelo 20
invention 20
cheek 20
awake 20
pursue 20


In [76]:
learning_rate=1e-4
reg=0.1
xmax=100
alpha=0.75
epochs=10000
sentences = sentences_small
word2idx = word2idx_small
D=100
context_sz = 10
V = len(word2idx)

In [77]:
X = np.zeros((V, V))
N = len(sentences)
print("number of sentences to process:", N)
it = 0
for sentence in sentences:
    it += 1
    if it % 10000 == 0:
        print("processed", it, "/", N)
    n = len(sentence)
    for i in range(n):
                    # i is not the word index!!!
                    # j is not the word index!!!
                    # i just points to which element of the sequence (sentence) we're looking at
        wi = sentence[i]

        start = max(0, i - context_sz)
        end = min(n, i + context_sz)

                    # we can either choose only one side as context, or both
                    # here we are doing both

                    # make sure "start" and "end" tokens are part of some context
                    # otherwise their f(X) will be 0 (denominator in bias update)
        if i - context_sz < 0:
            points = 1.0 / (i + 1)
            X[wi,0] += points
            X[0,wi] += points
        if i + context_sz > n:
            points = 1.0 / (n - i)
            X[wi,1] += points
            X[1,wi] += points

                    # left side
        for j in range(start, i):
            wj = sentence[j]
            points = 1.0 / (i - j) # this is +ve
            X[wi,wj] += points
            X[wj,wi] += points

                    # right side
        for j in range(i + 1, end):
            wj = sentence[j]
            points = 1.0 / (j - i) # this is +ve
            X[wi,wj] += points
            X[wj,wi] += points

            # save the cc matrix because it takes forever to create
np.save("cc_matrix_brown.npy", X)
print("max in X:", X.max())

        # weighting
fX = np.zeros((V, V))
fX[X < xmax] = (X[X < xmax] / float(xmax)) ** alpha
fX[X >= xmax] = 1

print("max in f(X):", fX.max())

        # target
logX = np.log(X + 1)

print("max in log(X):", logX.max())


number of sentences to process: 57013
processed 10000 / 57013
processed 20000 / 57013
processed 30000 / 57013
processed 40000 / 57013
processed 50000 / 57013
max in X: 194622.10793709665
max in f(X): 1.0
max in log(X): 12.17882018750701


In [78]:
# initialize weights
W = np.random.randn(V, D) / np.sqrt(V + D)
b = np.zeros(V)
U = np.random.randn(V, D) / np.sqrt(V + D)
c = np.zeros(V)
mu = logX.mean()


costs = []
sentence_indexes = range(len(sentences))
for epoch in range(epochs):
    delta = W.dot(U.T) + b.reshape(V, 1) + c.reshape(1, V) + mu - logX
    cost = ( fX * delta * delta ).sum()
    costs.append(cost)
    print("epoch:", epoch, "cost:", cost)

            
                # gradient descent method
                # update W
                # oldW = W.copy()
    for i in range(V):
                    # for j in range(V):
                    #     W[i] -= learning_rate*fX[i,j]*(W[i].dot(U[j]) + b[i] + c[j] + mu - logX[i,j])*U[j]
        W[i] -= learning_rate*(fX[i,:]*delta[i,:]).dot(U)
    W -= learning_rate*reg*W
                # print "updated W"

                # update b
    for i in range(V):
                    # for j in range(V):
                    #     b[i] -= learning_rate*fX[i,j]*(W[i].dot(U[j]) + b[i] + c[j] + mu - logX[i,j])
        b[i] -= learning_rate*fX[i,:].dot(delta[i,:])
                # b -= learning_rate*reg*b
                # print "updated b"

                # update U
    for j in range(V):
                    # for i in range(V):
                    #     U[j] -= learning_rate*fX[i,j]*(W[i].dot(U[j]) + b[i] + c[j] + mu - logX[i,j])*W[i]
        U[j] -= learning_rate*(fX[:,j]*delta[:,j]).dot(W)
    U -= learning_rate*reg*U
                # print "updated U"

                # update c
    for j in range(V):
                    # for i in range(V):
                    #     c[j] -= learning_rate*fX[i,j]*(W[i].dot(U[j]) + b[i] + c[j] + mu - logX[i,j])
        c[j] -= learning_rate*fX[:,j].dot(delta[:,j])
                # c -= learning_rate*reg*c
                # print "updated c"


epoch: 0 cost: 926289.7059441044
epoch: 1 cost: 692891.5731272013
epoch: 2 cost: 548527.6082615067
epoch: 3 cost: 454324.1307742888
epoch: 4 cost: 389597.6439497903
epoch: 5 cost: 342977.4710229017
epoch: 6 cost: 307984.2947128272
epoch: 7 cost: 280783.34338092885
epoch: 8 cost: 259015.77887802824
epoch: 9 cost: 241174.69756959984
epoch: 10 cost: 226262.1915593294
epoch: 11 cost: 213595.010554036
epoch: 12 cost: 202690.90567842204
epoch: 13 cost: 193200.03687644986
epoch: 14 cost: 184862.29739709274
epoch: 15 cost: 177479.98802893955
epoch: 16 cost: 170899.84674692657
epoch: 17 cost: 165000.93809659363
epoch: 18 cost: 159686.3075568659
epoch: 19 cost: 154877.11239328125
epoch: 20 cost: 150508.41672621787
epoch: 21 cost: 146526.12694986968
epoch: 22 cost: 142884.7225179303
epoch: 23 cost: 139545.55055334736
epoch: 24 cost: 136475.52617296783
epoch: 25 cost: 133646.12885400094
epoch: 26 cost: 131032.61766758829
epoch: 27 cost: 128613.41035053949
epoch: 28 cost: 126369.58649157792
epoch: 

epoch: 238 cost: 78210.65622136068
epoch: 239 cost: 78167.62678815694
epoch: 240 cost: 78124.36686775001
epoch: 241 cost: 78080.86718841126
epoch: 242 cost: 78037.11844213991
epoch: 243 cost: 77993.11128464171
epoch: 244 cost: 77948.83633541515
epoch: 245 cost: 77904.28417795083
epoch: 246 cost: 77859.44536005118
epoch: 247 cost: 77814.310394265
epoch: 248 cost: 77768.86975844784
epoch: 249 cost: 77723.11389644782
epoch: 250 cost: 77677.03321891918
epoch: 251 cost: 77630.61810427083
epoch: 252 cost: 77583.85889974775
epoch: 253 cost: 77536.74592265434
epoch: 254 cost: 77489.2694617179
epoch: 255 cost: 77441.41977860233
epoch: 256 cost: 77393.1871095676
epoch: 257 cost: 77344.56166728362
epoch: 258 cost: 77295.53364280668
epoch: 259 cost: 77246.0932077047
epoch: 260 cost: 77196.23051635953
epoch: 261 cost: 77145.93570842975
epoch: 262 cost: 77095.19891148648
epoch: 263 cost: 77044.01024382509
epoch: 264 cost: 76992.35981745676
epoch: 265 cost: 76940.23774127701
epoch: 266 cost: 76887.63

epoch: 473 cost: 57513.7475802691
epoch: 474 cost: 57449.861909787076
epoch: 475 cost: 57386.534235213454
epoch: 476 cost: 57323.75770909703
epoch: 477 cost: 57261.52551939375
epoch: 478 cost: 57199.83089367038
epoch: 479 cost: 57138.667103035965
epoch: 480 cost: 57078.02746580825
epoch: 481 cost: 57017.90535092416
epoch: 482 cost: 56958.294181098296
epoch: 483 cost: 56899.18743574329
epoch: 484 cost: 56840.57865365747
epoch: 485 cost: 56782.4614354868
epoch: 486 cost: 56724.82944597322
epoch: 487 cost: 56667.67641599413
epoch: 488 cost: 56610.99614440461
epoch: 489 cost: 56554.78249968876
epoch: 490 cost: 56499.029421426756
epoch: 491 cost: 56443.73092159121
epoch: 492 cost: 56388.881085675304
epoch: 493 cost: 56334.47407366208
epoch: 494 cost: 56280.50412084648
epoch: 495 cost: 56226.965538510565
epoch: 496 cost: 56173.85271446774
epoch: 497 cost: 56121.160113475045
epoch: 498 cost: 56068.88227752747
epoch: 499 cost: 56017.013826035334
epoch: 500 cost: 55965.54945589917
epoch: 501 co

epoch: 706 cost: 49882.059879259046
epoch: 707 cost: 49864.05672464472
epoch: 708 cost: 49846.10981701127
epoch: 709 cost: 49828.21869890662
epoch: 710 cost: 49810.38291651265
epoch: 711 cost: 49792.6020196094
epoch: 712 cost: 49774.87556154296
epoch: 713 cost: 49757.20309918862
epoch: 714 cost: 49739.584192920316
epoch: 715 cost: 49722.01840657614
epoch: 716 cost: 49704.50530742571
epoch: 717 cost: 49687.044466138366
epoch: 718 cost: 49669.63545675066
epoch: 719 cost: 49652.27785663571
epoch: 720 cost: 49634.97124647098
epoch: 721 cost: 49617.71521020979
epoch: 722 cost: 49600.50933504756
epoch: 723 cost: 49583.35321139396
epoch: 724 cost: 49566.246432843305
epoch: 725 cost: 49549.18859614441
epoch: 726 cost: 49532.1793011715
epoch: 727 cost: 49515.21815089631
epoch: 728 cost: 49498.30475135946
epoch: 729 cost: 49481.43871164199
epoch: 730 cost: 49464.61964383734
epoch: 731 cost: 49447.84716302614
epoch: 732 cost: 49431.120887246114
epoch: 733 cost: 49414.44043746828
epoch: 734 cost: 

epoch: 939 cost: 46560.56249747859
epoch: 940 cost: 46548.22094613289
epoch: 941 cost: 46535.88586069779
epoch: 942 cost: 46523.55717864192
epoch: 943 cost: 46511.23483837634
epoch: 944 cost: 46498.91877925413
epoch: 945 cost: 46486.60894156798
epoch: 946 cost: 46474.30526655007
epoch: 947 cost: 46462.00769636874
epoch: 948 cost: 46449.716174129404
epoch: 949 cost: 46437.43064387052
epoch: 950 cost: 46425.15105056443
epoch: 951 cost: 46412.87734011427
epoch: 952 cost: 46400.60945935346
epoch: 953 cost: 46388.3473560438
epoch: 954 cost: 46376.09097887343
epoch: 955 cost: 46363.840277457115
epoch: 956 cost: 46351.59520233185
epoch: 957 cost: 46339.355704958296
epoch: 958 cost: 46327.121737716436
epoch: 959 cost: 46314.89325390644
epoch: 960 cost: 46302.67020774526
epoch: 961 cost: 46290.45255436548
epoch: 962 cost: 46278.24024981341
epoch: 963 cost: 46266.033251047986
epoch: 964 cost: 46253.831515938466
epoch: 965 cost: 46241.63500326296
epoch: 966 cost: 46229.44367270671
epoch: 967 cost

...

epoch: 9689 cost: 13131.081442377701
epoch: 9690 cost: 13130.314629778664
epoch: 9691 cost: 13129.547962475903
epoch: 9692 cost: 13128.781440427876
epoch: 9693 cost: 13128.015063592848
epoch: 9694 cost: 13127.248831929193
epoch: 9695 cost: 13126.482745395235
epoch: 9696 cost: 13125.716803949304
epoch: 9697 cost: 13124.951007550038
epoch: 9698 cost: 13124.185356155584
epoch: 9699 cost: 13123.41984972441
epoch: 9700 cost: 13122.654488215065
epoch: 9701 cost: 13121.889271585944
epoch: 9702 cost: 13121.124199795531
epoch: 9703 cost: 13120.359272802345
epoch: 9704 cost: 13119.594490564854
epoch: 9705 cost: 13118.829853041594
epoch: 9706 cost: 13118.065360191167
epoch: 9707 cost: 13117.301011972026
epoch: 9708 cost: 13116.536808342928
epoch: 9709 cost: 13115.772749262234
epoch: 9710 cost: 13115.008834688764
epoch: 9711 cost: 13114.245064581039
epoch: 9712 cost: 13113.481438897677
epoch: 9713 cost: 13112.717957597362
epoch: 9714 cost: 13111.954620638835
epoch: 9715 cost: 13111.191427980715
ep

epoch: 9912 cost: 12963.604369225764
epoch: 9913 cost: 12962.868955326278
epoch: 9914 cost: 12962.133677823625
epoch: 9915 cost: 12961.398536679622
epoch: 9916 cost: 12960.66353185612
epoch: 9917 cost: 12959.928663315004
epoch: 9918 cost: 12959.19393101809
epoch: 9919 cost: 12958.459334927233
epoch: 9920 cost: 12957.724875004445
epoch: 9921 cost: 12956.990551211473
epoch: 9922 cost: 12956.256363510389
epoch: 9923 cost: 12955.52231186311
epoch: 9924 cost: 12954.7883962315
epoch: 9925 cost: 12954.05461657754
epoch: 9926 cost: 12953.320972863383
epoch: 9927 cost: 12952.587465050843
epoch: 9928 cost: 12951.85409310196
epoch: 9929 cost: 12951.120856978907
epoch: 9930 cost: 12950.387756643522
epoch: 9931 cost: 12949.654792057985
epoch: 9932 cost: 12948.92196318433
epoch: 9933 cost: 12948.189269984747
epoch: 9934 cost: 12947.456712421137
epoch: 9935 cost: 12946.724290455739
epoch: 9936 cost: 12945.99200405077
epoch: 9937 cost: 12945.259853168276
epoch: 9938 cost: 12944.527837770338
epoch: 993

epoch: 10132 cost: 12805.033660144967
epoch: 10133 cost: 12804.327359166402
epoch: 10134 cost: 12803.62118653095
epoch: 10135 cost: 12802.915142203557
epoch: 10136 cost: 12802.209226149065
epoch: 10137 cost: 12801.503438332564
epoch: 10138 cost: 12800.79777871897
epoch: 10139 cost: 12800.092247273204
epoch: 10140 cost: 12799.386843960236
epoch: 10141 cost: 12798.68156874518
epoch: 10142 cost: 12797.976421593004
epoch: 10143 cost: 12797.271402468683
epoch: 10144 cost: 12796.566511337305
epoch: 10145 cost: 12795.861748163852
epoch: 10146 cost: 12795.157112913606
epoch: 10147 cost: 12794.452605551443
epoch: 10148 cost: 12793.748226042411
epoch: 10149 cost: 12793.043974351795
epoch: 10150 cost: 12792.339850444649
epoch: 10151 cost: 12791.635854286049
epoch: 10152 cost: 12790.9319858412
epoch: 10153 cost: 12790.228245075268
epoch: 10154 cost: 12789.524631953434
epoch: 10155 cost: 12788.821146440798
epoch: 10156 cost: 12788.1177885027
epoch: 10157 cost: 12787.4145581042
epoch: 10158 cost: 12

epoch: 10350 cost: 12654.037203347969
epoch: 10351 cost: 12653.358073594853
epoch: 10352 cost: 12652.679064842441
epoch: 10353 cost: 12652.000177058631
epoch: 10354 cost: 12651.321410211027
epoch: 10355 cost: 12650.64276426728
epoch: 10356 cost: 12649.964239195311
epoch: 10357 cost: 12649.285834962624
epoch: 10358 cost: 12648.607551537072
epoch: 10359 cost: 12647.92938888645
epoch: 10360 cost: 12647.251346978415
epoch: 10361 cost: 12646.573425780878
epoch: 10362 cost: 12645.895625261443
epoch: 10363 cost: 12645.217945388094
epoch: 10364 cost: 12644.540386128587
epoch: 10365 cost: 12643.862947450707
epoch: 10366 cost: 12643.185629322277
epoch: 10367 cost: 12642.508431711183
epoch: 10368 cost: 12641.831354585407
epoch: 10369 cost: 12641.154397912655
epoch: 10370 cost: 12640.47756166089
epoch: 10371 cost: 12639.800845798021
epoch: 10372 cost: 12639.124250291874
epoch: 10373 cost: 12638.447775110464
epoch: 10374 cost: 12637.771420221694
epoch: 10375 cost: 12637.095185593469
epoch: 10376 co

KeyboardInterrupt: 

In [ ]:
W1 = W
W2 = U.T

In [ ]:
W2.shape

In [ ]:
We = (W1+W2.T)/2

In [ ]:
idx2word = {i:w for w,i in word2idx.items()}

In [ ]:
w1='king'
w2='man'
w3='woman'
V, D = We.shape
p1 = We[word2idx[w1]]
n1 = We[word2idx[w2]]
n2 = We[word2idx[w3]]
v0 = p1 - n1 + n2

for dist in ('euclidean', 'cosine'):
    distances = pairwise_distances(v0.reshape(1, D), We, metric=dist).reshape(V)
        # idx = distances.argmin()
        # best_word = idx2word[idx]
    print(distances)
    idx = distances.argsort()[:4]
    print(idx)
    best_idx = -1
    keep_out = [word2idx[w] for w in (w1, w2, w3)]
    for i in idx:
        if i not in keep_out:
            best_idx = i
            break
    best_word = idx2word[best_idx]


    print("closest match by", dist, "distance:", best_word)
    print(w1, "-", w2, "=", best_word, "-", w3)

In [ ]:
arrays = [W, T]
        np.savez('glove_model_50.npz', *arrays)